In [1]:
# Import necessary libraries

from pytrends.request import TrendReq
import pandas as pd
import requests
import lxml
from datetime import datetime
import credential
import pymysql

In [2]:
# Input the target country

val = input("Enter target country (use correct capitalization and white space, empty this field if you are pushing labeled table): ")

Enter target country (use correct capitalization and white space, empty this field if you are pushing labeled table): United States


In [3]:
# Input date of interest
date_of_interest = input("Enter the date of the data (YYY-MM-DD), insert this if you intend to only update database, too:")

Enter the date of the data (YYY-MM-DD), insert this if you intend to only update database, too:2022-03-27


In [4]:
# Input the number of passed data points or offset if the day change time is missed

offset = input("Enter the number of offset if the day change time is missed: ")

Enter the number of offset if the day change time is missed: 1


In [5]:
# Input the number of data rows

rows = input("Enter the number of data rows available: ")

Enter the number of data rows available: 20


In [6]:
# Input country data to be pushed to database after being labeled

val_labeled = input("Labeled country data to be pushed, if there is any (use correct capitalization and white space): ")

Labeled country data to be pushed, if there is any (use correct capitalization and white space): 


In [7]:
# Create list of countries in need of labeling
to_label = ['Indonesia',
            'United States',
            'Australia',
            'United Kingdom',
            'Singapore']

In [8]:
# Create dictionary of country names for mapping

ctr = {'Indonesia': 'indonesia',
      'Singapore': 'singapore',
      'United States': 'united_states',
      'United Kingdom': 'united_kingdom',
      'Australia': 'australia',
      'India': 'india',
      'South Korea': 'south_korea',
      'Russia': 'russia',
      'Germany': 'germany',
      'France': 'france',
      'Japan': 'japan'}

In [9]:
def get_trends(val, date_of_interest, offset, rows):
    
    # Create dictionary of country names for mapping

    ctr = {'Indonesia': 'indonesia',
          'Singapore': 'singapore',
          'United States': 'united_states',
          'United Kingdom': 'united_kingdom',
          'Australia': 'australia',
          'India': 'india',
          'South Korea': 'south_korea',
          'Russia': 'russia',
          'Germany': 'germany',
          'France': 'france',
          'Japan': 'japan'}

# Instantiate the pytrends object

    pytrends = TrendReq()

# Get the trending searches

    df = pytrends.trending_searches(pn=ctr[val])

# Get the date of interest and put it in a column

    df['log_date'] = date_of_interest
    
# Handle the offset
    
    offst = 0
    if offset !="" :
        offst = int(offset)
        
    df = df.tail(20-offst)

# Handle the data rows number just in case the number of rows is not 20
    
    rows_to_get = 0
    
    if rows !="" :
        if len(df) >= int(rows):
            rows_to_get = int(rows)
        else:
            rows_to_get = len(df)
        
    df = df.head(rows_to_get)

# Get the term rank numbers based on order of the data and put them in a column

    df['term_rank'] = range(1, len(df)+1)

# Rename the term column

    df = df.rename(columns={0: 'term'})

#  Create an topic column
    
    df['topic'] = 'No Topic'

# Create the country column

    df['country'] = val

# Reorder the columns

    df = df[['log_date', 'country', 'term', 'topic', 'term_rank']]

# Export to Excel

    path = "C:/Users/Nugroho Budianggoro/Documents/NadiData/1282/data/"
    filename = path + "trending_searches_" + str(ctr[val]) + "_" + date_of_interest + ".xlsx"
    df.to_excel(filename, index=False)
 
    return df

In [10]:
# Define function to update database

def update_db(df, no_topic):
    
    if no_topic == False:
    # Map the topic abbreviations to normal topic titles

        df_news_cat = pd.read_csv("news_cat.csv", sep=";")

        df['topic'] = df['topic'].map(df_news_cat.set_index('topic')['full_topic'])

    # Duplicate dataframe and then change the term values into topic values

        df_topics = df.copy()
        df_topics['term'] = "TOPIC " + df_topics['topic']

    # Append the dataframe of topics into the original data frame

        df = df.append(df_topics, ignore_index=True)
    
# Establish connection to the database

    connection = pymysql.connect(host=credential.host,
                                user=credential.user,
                                password=credential.password,
                                db=credential.database)

    cursor = connection.cursor()

    # Get number of rows before addition

    sql = "SELECT * FROM `trending_search`"
    count_before = cursor.execute(sql)

# Add the new rows to the table while generating log about how many rows were added

    try:
        cols = "`,`".join([str(i) for i in df.columns.tolist()])
        for i, row in df.iterrows():
            sql = "INSERT INTO `trending_search` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
            cursor.execute(sql, tuple(row))
            connection.commit()
        sql = "SELECT * FROM `trending_search`"
        count_after = cursor.execute(sql)
        count_addition = count_after - count_before
        text = "Succesfully added " + str(count_addition) + " rows of data."
        print(text)

    except pymysql.Error as e:
        print(e)

    cursor.close()
    connection.close()

In [11]:
# Create function for processing between the in need of label and no need for label countries

if val !="" and val not in to_label :
    df = get_trends(val, date_of_interest, offset, rows)
    no_topic = True
    update_db(df, no_topic)
    
elif val !="" and val in to_label:
    get_trends(val, date_of_interest, offset, rows)
    
elif val_labeled != "" :
    country_filename = ctr[val_labeled]
    path = "C:/Users/Nugroho Budianggoro/Documents/NadiData/1282/data/"
    df = pd.read_excel(path + "trending_searches_" + country_filename + "_" + date_of_interest + ".xlsx")
    no_topic = False
    update_db(df, no_topic)